In [390]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
from scipy.spatial import cKDTree
from pyproj import Proj, transform
from pyproj import Transformer
from tqdm import tqdm

def transform_5179_to_4326(x, y):
    # Transformer 객체 생성
    transformer = Transformer.from_crs("epsg:5179", "epsg:4326", always_xy=True)
    # 좌표 변환
    lon, lat = transformer.transform(x, y)
    return lat, lon

def transform_2097_to_4326(x, y):
    # Transformer 객체 생성
    transformer = Transformer.from_crs("epsg:2097", "epsg:4326", always_xy=True)
    # 좌표 변환
    lon, lat = transformer.transform(x, y)
    return lat, lon

# 1. 격자 구조 인구밀도 데이터 -> 중심점 계산
def calculate_centroids(grid_df):
    grid_df['centroid'] = grid_df.geometry.centroid
    return grid_df

# 2. 좌표 체계 맞추기
def align_crs(grid_df, shops_gdf):
    # 격자 데이터 CRS 설정 (EPSG:4326 사용)
    if grid_df.crs is None:
        grid_df = grid_df.set_crs(epsg=4326, allow_override=True)
    
    # 상점 데이터 CRS 설정 (EPSG:4326 사용)
    if shops_gdf.crs is None:
        shops_gdf = shops_gdf.set_crs(epsg=4326, allow_override=True)
    
    # 상점 데이터를 격자 데이터의 CRS로 변환
    shops_gdf = shops_gdf.to_crs(grid_df.crs)
    
    return grid_df, shops_gdf


# 3. 격자 중심점과 상점 위치 간의 거리 계산
def calculate_accessibility(grid_df, shops_gdf, buffer_distance):
    grid_df['accessibility'] = 0
    
    # grid_df 중심점과 shops_gdf의 좌표를 numpy 배열로 변환
    centroids = np.array([[point.x, point.y] for point in grid_df.centroid])
    shop_coords = np.array([[point.x, point.y] for point in shops_gdf.geometry])
    
    print(f"Centroids shape: {centroids.shape}")
    print(f"Shop coordinates shape: {shop_coords.shape}")
    
    # array가 비어있는지 확인
    if centroids.shape[0] == 0 or shop_coords.shape[0] == 0:
        raise ValueError("Centroid or shop coordinates array is empty")
    
    # KD-Tree를 사용하여 상점 위치에 대한 거리 계산
    tree = cKDTree(shop_coords)
    
    # 각각의 인구 밀도 중심점에 대해, 근접한 상점 개수 계산
    for centroid, index in zip(centroids, grid_df.index):
        indices = tree.query_ball_point(centroid, buffer_distance)
        grid_df.at[index, 'accessibility'] = len(indices)
    
    return grid_df

def calculate_ppr_for_shops(grid_df, shops_gdf, buffer_distance):
    shops_gdf['PPR'] = 0
    
    # grid_df 중심점과 shops_gdf의 좌표를 numpy 배열로 변환
    centroids = np.array([[point.x, point.y] for point in grid_df['centroid']])
    shop_coords = np.array([[point.x, point.y] for point in shops_gdf.geometry])
    
    # KD-Tree를 사용하여 인구 위치에 대한 거리 계산
    tree = cKDTree(centroids)
    
    # 각각의 상점 위치에 대해, 근접한 격자의 인구 밀도 합계 계산
    for shop_idx, shop in shops_gdf.iterrows():
        indices = tree.query_ball_point([shop.geometry.x, shop.geometry.y], buffer_distance)
        ppr_value = grid_df.iloc[indices]['val'].sum()
        shops_gdf.at[shop_idx, 'PPR'] = 1/ppr_value
    
    return shops_gdf

def calculate_accessibility_for_grids(grid_df, shops_gdf, buffer_distance):
    grid_df['accessibility'] = 0
    
    # grid_df 중심점과 shops_gdf의 좌표를 numpy 배열로 변환
    centroids = np.array([[point.x, point.y] for point in grid_df.centroid])
    shop_coords = np.array([[point.x, point.y] for point in shops_gdf.geometry])
    shop_ppr_values = shops_gdf['PPR'].values
    
    # KD-Tree를 사용하여 상점 위치에 대한 거리 계산
    tree = cKDTree(shop_coords)
    
    # 각각의 인구 밀도(중심점)에 대해, buffer distance 내에 있는 상점들의 PPR 합계 계산
    for grid_idx, centroid in enumerate(centroids):
        indices = tree.query_ball_point(centroid, buffer_distance)
        accessibility_value = shop_ppr_values[indices].sum()
        grid_df.at[grid_idx, 'accessibility'] = accessibility_value
    
    return grid_df

In [392]:
import os

current_path = os.getcwd()
data_path = os.path.join(current_path, 'data')

# 데이터 로드 및 처리 예시
grid_file = os.path.join(data_path, '(B100)국토통계_인구정보-총 인구 수(전체)-(격자) 1KM_서울특별시_202404/nlsp_020001001.shp')  # 격자 데이터 파일 경로
shops_file = os.path.join(data_path,'서울시 대규모점포 인허가 정보.shp')  # 상점 데이터 파일 경로
buffer_distance = 5000  # 분석에 사용할 버퍼 거리 (단위: 미터)

# 격자 및 상점 데이터 불러오기
grid_df = gpd.read_file(grid_file)
shops_gdf = gpd.read_file(shops_file, encoding='ISO-8859-1')

# column 명 변환
x_name = shops_gdf.columns[23]
y_name = shops_gdf.columns[24]
shops_gdf = shops_gdf.rename(columns={x_name: 'x', y_name: 'y'})
shops_gdf['geometry'] = shops_gdf.apply(lambda x: Point((float(x.x), float(x.y))), axis=1)

In [393]:
# 결측치 제거
shops_gdf = shops_gdf.dropna(subset=['x', 'y'])
grid_df = grid_df.dropna(subset=['lbl'])
grid_df = grid_df[grid_df['val'] != 0]


In [394]:
# 격자 중심점 계산
grid_df = calculate_centroids(grid_df)

# 좌표 체계 맞추기
grid_df['centroid'] = grid_df['centroid'].apply(lambda geom: transform_5179_to_4326(geom.centroid.x, geom.centroid.y))

tqdm.pandas()
shops_gdf[['x', 'y']] = shops_gdf['geometry'].progress_apply(lambda geom: pd.Series(transform_2097_to_4326(geom.centroid.x, geom.centroid.y)))
shops_gdf['geometry'] = shops_gdf.apply(lambda x: Point((float(x.x), float(x.y))), axis=1)

shops_gdf

100%|██████████| 922/922 [00:04<00:00, 188.25it/s]


,ê°ë°©ìì,ê´ë¦¬ë²í,ì¸íê°ì,ì¸íê°_1,ìì ìí,ìì ì_1,ìì¸ìì,ìì¸ì_1,íì ì¼ì,í´ì ìì,...,ëë¡ëª_1,ì¬ì ì¥ë,ìµì¢ ìì,ë°ì´í°ê,ë°ì´í_1,ì íêµ¬ë,x,y,ì í¬êµ¬ë,geometry
0,3170000,2.013317e+18,2013-09-06,NaN,1,ìì /ì ì,1,ì ììì,None,None,...,153-801,ë§ë¦¬ì¤ìì¸ë  1ê´,2023/02/28 20:56:47.000,U,2022/12/03 00:03:00.000,ì¼íì¼í°,37.478189,126.884201,None,POINT (37.47819 126.8842)
1,3170000,2.012317e+18,2012-09-13,NaN,1,ìì /ì ì,1,ì ììì,None,None,...,None,ë§ë¦¬ì¤ìì¸ë  3ê´,2023/02/28 21:05:56.000,U,2022/12/03 00:03:00.000,ì¼íì¼í°,37.478456,126.882126,None,POINT (37.47846 126.88213)
2,3140000,2.012314e+18,2012-04-05,NaN,1,ìì /ì ì,1,ì ììì,None,None,...,None,ë¡¯ë°ìí¼ ëª©ë2ì ,2023/03/09 09:25:54.000,U,2022/12/02 23:01:00.000,êµ¬ë¶ìì,37.534359,126.870059,None,POINT (37.53436 126.87006)
3,3180000,2.001318e+18,1999-05-31,NaN,1,ìì /ì ì,1,ì ììì,None,None,...,150-723,ìë±í¬ì íµìê°,2023/03/09 15:28:54.000,U,2022/12/02 23:01:00.000,ê·¸ ë°ì ëê·ëª¨ì í¬,37.522476,126.891063,None,POINT (37.52248 126.89106)
4,3210000,2.012321e+18,2012-04-24,NaN,1,ìì /ì ì,1,ì ììì,None,None,...,137-832,ë¡¯ë°ìí¼ ë°©ë°°2ì ,2023/03/13 17:40:32.000,U,2022/12/02 23:05:00.000,ëíë§í¸,37.491268,126.986414,None,POINT (37.49127 126.98641)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,3040000,2.008304e+18,20080822,NaN,1,ìì /ì ì,1,ì ììì,None,None,...,143854,ë¡¯ë°ë°±íì  ê±´ëì¤íìí°ì ,2022/05/13 14:43:47.000,U,2021/12/04 23:05:00.000,ë°±íì ,37.538789,127.068247,None,POINT (37.53879 127.06825)
1000,3040000,2.001304e+18,1998-03-23,NaN,1,ìì /ì ì,1,ì ììì,None,None,...,143-721,ë¡¯ë°ì¼í(ì£¼)ë¡¯ë°ë§í¸ ê°ë³ì ,2023/04/10 10:48:38.000,U,2022/12/03 23:02:00.000,ëíë§í¸,37.535700,127.092875,None,POINT (37.5357 127.09287)
1001,3070000,2.012307e+18,2017-04-27,NaN,3,íì,3,íì ì²ë¦¬,2024-04-08,None,...,136-130,ë¡¯ë°ë§ì¼999 íìê³¡ì ,2024/04/17 13:37:54.000,U,2023/12/03 23:09:00.000,êµ¬ë¶ìì,37.608779,127.016260,None,POINT (37.60878 127.01626)
1002,3070000,2.012307e+18,2012-05-21,NaN,1,ìì /ì ì,1,ì ììì,None,None,...,136-140,ë¡¯ë°ë§ì´ìí¼ ì¥ìì ,2023/05/19 10:42:09.000,U,2022/12/04 22:01:00.000,êµ¬ë¶ìì,37.622935,127.045570,None,POINT (37.62294 127.04557)


In [395]:
grid_df

,gid,lbl,val,geometry,centroid
0,ë¤ì¬6453,8975.00,8975.0,"POLYGON ((964000 1953000, 964000 1954000, 9650...","(37.580200485292174, 127.09794163346876)"
1,ë¤ì¬5651,12141.00,12141.0,"POLYGON ((956000 1951000, 956000 1952000, 9570...","(37.56183067511147, 127.00745844564479)"
2,ë¤ì¬6157,28042.00,28042.0,"POLYGON ((961000 1957000, 961000 1958000, 9620...","(37.61613289244262, 127.06375529148733)"
3,ë¤ì¬5858,22857.00,22857.0,"POLYGON ((958000 1958000, 958000 1959000, 9590...","(37.625015511173096, 127.0297064353994)"
4,ë¤ì¬5347,12053.00,12053.0,"POLYGON ((953000 1947000, 953000 1948000, 9540...","(37.52563120840341, 126.97374482365522)"
...,...,...,...,...,...
604,ë¤ì¬4549,12779.00,12779.0,"POLYGON ((945000 1949000, 945000 1950000, 9460...","(37.54321927164485, 126.88306227047312)"
605,ë¤ì¬5142,40229.00,40229.0,"POLYGON ((951000 1942000, 951000 1943000, 9520...","(37.480461946348484, 126.9514409502969)"
606,ë¤ì¬5548,11214.00,11214.0,"POLYGON ((955000 1948000, 955000 1949000, 9560...","(37.53474316603645, 126.99631802960324)"
607,ë¤ì¬5950,12661.00,12661.0,"POLYGON ((959000 1950000, 959000 1951000, 9600...","(37.55295420646501, 127.04148102816576)"


In [389]:
# 좌표 체계 맞추기
# grid_df, shops_gdf = align_crs(grid_df, shops_gdf)

# 접근성 분석
grid_df = calculate_accessibility(grid_df, shops_gdf, buffer_distance)

# 결과 출력
print(grid_df.head())

AttributeError: 'tuple' object has no attribute 'x'

In [375]:
# 4. 상점별 PPR 값 계산 (2SFCA 기법 - 1단계)
shops_gdf = calculate_ppr_for_shops(grid_df, shops_gdf, buffer_distance)

print(shops_gdf['PPR'])
print(shops_gdf.shape)

/tmp/ipykernel_2424127/1933376995.py:84: RuntimeWarning: divide by zero encountered in scalar divide
  shops_gdf.at[shop_idx, 'PPR'] = 1/ppr_value
/tmp/ipykernel_2424127/1933376995.py:84: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'inf' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  shops_gdf.at[shop_idx, 'PPR'] = 1/ppr_value
/tmp/ipykernel_2424127/1933376995.py:84: RuntimeWarning: divide by zero encountered in scalar divide
  shops_gdf.at[shop_idx, 'PPR'] = 1/ppr_value


0       inf
1       inf
2       inf
3       inf
4       inf
       ... 
999     inf
1000    inf
1001    inf
1002    inf
1003    inf
Name: PPR, Length: 922, dtype: float64
(922, 28)


In [376]:
# 5. 격자별 접근성 지수 계산 (2SFCA 기법 - 2단계)
# 격자별 접근성 지수 계산
grid_df = calculate_accessibility_for_grids(grid_df, shops_gdf, buffer_distance)

print(grid_df.head())

          gid       lbl      val  \
0  ë¤ì¬6453   8975.00   8975.0   
1  ë¤ì¬5651  12141.00  12141.0   
2  ë¤ì¬6157  28042.00  28042.0   
3  ë¤ì¬5858  22857.00  22857.0   
4  ë¤ì¬5347  12053.00  12053.0   

                                            geometry  \
0  POLYGON ((964000 1953000, 964000 1954000, 9650...   
1  POLYGON ((956000 1951000, 956000 1952000, 9570...   
2  POLYGON ((961000 1957000, 961000 1958000, 9620...   
3  POLYGON ((958000 1958000, 958000 1959000, 9590...   
4  POLYGON ((953000 1947000, 953000 1948000, 9540...   

                                   centroid  accessibility  
0  (37.580200485292174, 127.09794163346876)            0.0  
1   (37.56183067511147, 127.00745844564479)            0.0  
2   (37.61613289244262, 127.06375529148733)            0.0  
3   (37.625015511173096, 127.0297064353994)            0.0  
4   (37.52563120840341, 126.97374482365522)            0.0  


In [377]:
for idx, row in grid_df.iterrows():
    if row['accessibility'] != 0:
        print(f"격자 {row['lbl']}의 접근성 지수: {row['accessibility']}")

### 서울시 동 경계 데이터

In [191]:
geojson_file = 'EMD_Seoul.geojson'
gdf = gpd.read_file(geojson_file)
gdf.head()

,BASE_DATE,ADM_DR_CD,ADM_DR_NM,OBJECTID,geometry
0,20200630,1101053,사직동,1,"MULTIPOLYGON (((126.97399 37.57823, 126.974 37..."
1,20200630,1101054,삼청동,2,"MULTIPOLYGON (((126.97714 37.59768, 126.9773 3..."
2,20200630,1101055,부암동,3,"MULTIPOLYGON (((126.96173 37.60714, 126.96182 ..."
3,20200630,1101056,평창동,4,"MULTIPOLYGON (((126.97509 37.63118, 126.97488 ..."
4,20200630,1101057,무악동,5,"MULTIPOLYGON (((126.95975 37.58001, 126.96006 ..."


In [192]:
# 좌표계 변환 
grid_df, gdf = align_crs(grid_df, gdf)

In [193]:
# 포인트가 속한 행정동 경계 찾기
def find_admin_dong(point, gdf):
    for idx, row in gdf.iterrows():
        if row['geometry'].contains(point):  # point가 폴리곤 내부에 있는지 확인
            return row['ADM_DR_NM']  # 행정동 이름 반환
    return None  # 행정동 경계에 포함되지 않으면 None을 반환

grid_df['District_Name'] = grid_df.apply(lambda row: find_admin_dong(row['centroid'], gdf), axis=1)

print(grid_df['District_Name'].head(10))

In [221]:
grid_df.tail()

,gid,lbl,val,geometry,centroid,accessibility,District_Name,Sales
705,ë¤ì¬6752,None,0.0,"POLYGON ((967000 1952000, 967000 1953000, 9680...",POINT (967500 1952500),0,None,0.0
706,ë¤ì¬7249,None,0.0,"POLYGON ((972000 1949000, 972000 1950000, 9730...",POINT (972500 1949500),0,None,0.0
707,ë¤ì¬4552,None,0.0,"POLYGON ((945000 1952000, 945000 1953000, 9460...",POINT (945500 1952500),0,상암동,8.0
708,ë¤ì¬6265,None,0.0,"POLYGON ((962000 1965000, 962000 1966000, 9630...",POINT (962500 1965500),0,상계1동,7.0
709,ë¤ì¬5259,None,0.0,"POLYGON ((952000 1959000, 952000 1960000, 9530...",POINT (952500 1959500),0,진관동,8.0


In [223]:
for idx, row in grid_df.iterrows():
    if pd.isna(row['District_Name']):
        print(f"Missing geometry is {row['centroid']}")
        print(f"Missing district name for grid {idx}")

Missing geometry is POINT (953500 1959500)
Missing district name for grid 30
Missing geometry is POINT (950500 1962500)
Missing district name for grid 50
Missing geometry is POINT (966500 1940500)
Missing district name for grid 54
Missing geometry is POINT (971500 1953500)
Missing district name for grid 62
Missing geometry is POINT (968500 1941500)
Missing district name for grid 101
Missing geometry is POINT (960500 1965500)
Missing district name for grid 119
Missing geometry is POINT (944500 1942500)
Missing district name for grid 120
Missing geometry is POINT (971500 1949500)
Missing district name for grid 146
Missing geometry is POINT (942500 1942500)
Missing district name for grid 162
Missing geometry is POINT (939500 1941500)
Missing district name for grid 175
Missing geometry is POINT (947500 1958500)
Missing district name for grid 191
Missing geometry is POINT (966500 1955500)
Missing district name for grid 216
Missing geometry is POINT (939500 1942500)
Missing district name for

### 서울 상권분석 (소비 매출) 데이터

In [208]:
sales_path = os.path.join(data_path, '서울시 상권분석서비스(소득소비-행정동).csv')
sales_data = pd.read_csv(sales_path, encoding='cp949')
# sales_data['Sales'] = None

# 2024-1분기 데이터만 사용
sales_data = sales_data[sales_data['기준_년분기_코드'] == 20241]

In [209]:
sales_data.head()

,기준_년분기_코드,행정동_코드,행정동_코드_명,월_평균_소득_금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,의료비_지출_총금액,교통_지출_총금액,교육_지출_총금액,유흥_지출_총금액,여가_문화_지출_총금액,기타_지출_총금액,음식_지출_총금액
1700,20241,11110680,창신2동,2112817,5,378781000,114626000,2615000,3824000,54984000,1872000,1625000,6167000,8797000,7498000,176773000
1701,20241,11110670,창신1동,2393308,5,885535000,150365000,56693000,1192000,227077000,754000,29247000,35303000,92980000,49143000,242781000
1702,20241,11170700,보광동,2545653,6,378073000,117293000,7390000,6950000,49139000,3287000,14383000,18357000,13263000,13958000,134053000
1703,20241,11140650,신당5동,3027979,7,916706000,164793000,22144000,13424000,135532000,38454000,4674000,55450000,107647000,17265000,357323000
1704,20241,11170510,후암동,3013677,7,732920000,308335000,12720000,10397000,36686000,11974000,30917000,7239000,22807000,28991000,262854000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2120,20241,11680545,압구정동,5808537,9,14896571000,769464000,1038252000,112970000,4482374000,88273000,787208000,407782000,3639012000,820108000,2751128000
2121,20241,11680521,논현1동,3277066,7,11309743000,820163000,133734000,165229000,5701705000,68482000,178445000,342037000,1492218000,418257000,1989473000
2122,20241,11710531,거여1동,2789100,6,619182000,152727000,4354000,17321000,103912000,24811000,51382000,37492000,33762000,31889000,161532000
2123,20241,11680740,일원2동,4553185,8,457834000,171029000,4146000,8779000,70858000,35009000,33790000,2593000,42557000,13130000,75943000


In [215]:
for idx, row in grid_df.iterrows():
    district_name = row['District_Name']
    if district_name in sales_data['행정동_코드_명'].values:
        grid_df.at[idx, 'Sales'] = sales_data[sales_data['행정동_코드_명']== district_name]['소득_구간_코드'].iloc[0]
    
    else:
        grid_df.at[idx, 'Sales'] = 0

In [216]:
grid_df['Sales'].head()
print((grid_df['Sales'] == 0).sum()) # 모든 행정 구역 동 소득 구간 정보 확인.

117


None
상계3·4동
종로5·6가동
None
None
None
None
종로1·2·3·4가동
상계3·4동
None
None
종로1·2·3·4가동
상계6·7동
None
None
상계3·4동
None
None
None
None
None
None
면목3·8동
None
None
None
상계3·4동
None
None
None
None
중계2·3동
None
None
None
상계3·4동
None
종로1·2·3·4가동
None
None
None
None
None
None
None
None
금호2·3가동
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [97]:
grid_df

,gid,lbl,val,geometry,centroid,accessibility,District_Name,Sales
0,ë¤ì¬609556,367.00,367.0,"POLYGON ((960900 1955600, 960900 1955700, 9610...",POINT (960950 1955650),0.000000,이문1동,7.0
1,ë¤ì¬624528,1144.00,1144.0,"POLYGON ((962400 1952800, 962400 1952900, 9625...",POINT (962450 1952850),0.000059,장안2동,6.0
2,ë¤ì¬625528,1369.00,1369.0,"POLYGON ((962500 1952800, 962500 1952900, 9626...",POINT (962550 1952850),0.000059,장안2동,6.0
3,ë¤ì¬616520,225.00,225.0,"POLYGON ((961600 1952000, 961600 1952100, 9617...",POINT (961650 1952050),0.000000,장안1동,6.0
4,ë¤ì¬615519,484.00,484.0,"POLYGON ((961500 1951900, 961500 1952000, 9616...",POINT (961550 1951950),0.000000,장안1동,6.0
...,...,...,...,...,...,...,...,...
1534,ë¤ì¬611558,None,NaN,"POLYGON ((961100 1955800, 961100 1955900, 9612...",POINT (961150 1955850),0.000000,이문1동,7.0
1535,ë¤ì¬623539,None,NaN,"POLYGON ((962300 1953900, 962300 1954000, 9624...",POINT (962350 1953950),0.000000,면목5동,5.0
1536,ë¤ì¬615533,None,NaN,"POLYGON ((961500 1953300, 961500 1953400, 9616...",POINT (961550 1953350),0.000097,전농2동,7.0
1537,ë¤ì¬598551,None,NaN,"POLYGON ((959800 1955100, 959800 1955200, 9599...",POINT (959850 1955150),0.000000,청량리동,6.0


### 서울 평균 연령 데이터

In [102]:
age_path = os.path.join(data_path, '평균연령_20240726021229.csv')
age_data = pd.read_csv(age_path, encoding='utf-8')
age_data.tail()

,동별(1),동별(2),동별(3),2024 2/4,2024 2/4.1,2024 2/4.2
448,합계,강동구,천호2동,46.2,45.7,46.6
449,합계,강동구,길동,46.2,45.5,46.9
450,합계,강동구,강일동,43.1,41.7,44.3
451,합계,강동구,상일1동,40.2,39.8,40.5
452,합계,강동구,상일2동,40.7,39.6,41.6


In [103]:
for idx, row in grid_df.iterrows():
    district_name = row['District_Name']
    if district_name in age_data['동별(3)'].values:
        grid_df.at[idx, 'Average_age'] = age_data[age_data['동별(3)']== district_name]['2024 2/4'].iloc[0]
    
    else:
        grid_df.at[idx, 'Average_age'] = 0

In [115]:
grid_df['Average_age'].head()
print((grid_df['Average_age'] == 0).sum()) # 모든 행정 구역 동 평균 연령 정보 확인.

0


### Nearest large store 계산

In [129]:
shops_gdf.head()

,ê°ë°©ìì,ê´ë¦¬ë²í,ì¸íê°ì,ì¸íê°_1,ìì ìí,ìì ì_1,ìì¸ìì,ìì¸ì_1,íì ì¼ì,í´ì ìì,...,ì¬ì ì¥ë,ìµì¢ ìì,ë°ì´í°ê,ë°ì´í_1,ì íêµ¬ë,ì¢íì ë,ì¢íì _1,ì í¬êµ¬ë,geometry,PPR
0,3050000,2.023305e+18,2023-03-14,None,1,ìì /ì ì,5,ìì ê°ìì ,None,None,...,GS THE FRESH ëëë¬¸ì¥ìì ,2023/03/14 09:57:17.000,I,2022/12/02 23:06:00.000,êµ¬ë¶ìì,206341.538777,452530.018812,None,POINT (962186.516 1952632.748),0.000029
1,3050000,2.023305e+18,2023-03-14,None,1,ìì /ì ì,1,ì ììì,None,None,...,GS THE FRESH ëëë¬¸ì¥ìì ,2023/07/10 09:11:32.000,U,2022/12/06 23:03:00.000,êµ¬ë¶ìì,206341.538777,452530.018812,None,POINT (962186.516 1952632.748),0.000029
2,3050000,2.000305e+18,19600629,None,1,ìì /ì ì,1,ì ììì,None,None,...,ê²½ëìì¥,2022/04/21 09:11:11.000,U,2021/12/03 22:03:00.000,ìì¥,203403.019535,452985.725420,None,POINT (959251.583 1953103.906),0.000056
3,3050000,2.022305e+18,2022-07-11,None,1,ìì /ì ì,1,ì ììì,None,None,...,ì²­ëë¦¬ì­ íìììì¸ ê·¸ë¼ìì íë...,2023/06/29 10:04:00.000,U,2022/12/07 00:01:00.000,ê·¸ ë°ì ëê·ëª¨ì í¬,203728.022004,452778.965235,None,POINT (959575.357 1952895.498),0.000057
4,3050000,2.013305e+18,20131127,None,1,ìì /ì ì,1,ì ììì,None,None,...,ë² ë¼ì²´ìº í¼ì¤ ìë¹ë´,2022/04/06 11:28:46.000,U,2021/12/04 00:08:00.000,ê·¸ ë°ì ëê·ëª¨ì í¬,204881.313219,454280.040556,None,POINT (960736.179 1954389.845),0.000047


In [ ]:
from geopy.distance import geodesic

# 새로운 컬럼을 추가할 빈 데이터프레임 생성
grid_df['Nearest_dist'] = float('inf')  # 초기값을 무한대로 설정

# 인구 밀도 데이터의 각 위치에 대해 가장 가까운 상권 찾기
for _, row in grid_df.iterrows():
    point = row['centroid']
    
    # 각 상권 위치와 인구 밀도 위치 간의 거리 계산
    for idx, store_row in shop_gdf.iterrows():
        store_point = (store_row['상권위도'], store_row['상권경도'])
        distance = geodesic(pop_point, store_point).kilometers
        
        # 가장 가까운 상권까지의 거리 업데이트
        if distance < store_data.loc[idx, '최단거리']:
            store_data.loc[idx, '최단거리'] = distance
